In [ ]:
#!pip install librosa

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import IPython.display as ipd 
import librosa
import librosa.display

In [ ]:
filename = '../input/urbansound8k/fold8/103076-3-0-0.wav'
plt.figure(figsize=(14,5))
data,sample_rate=librosa.load(filename)
librosa.display.waveplot(data,sr=sample_rate)
ipd.Audio(filename)

In [ ]:
from scipy.io import wavfile as wav
wave_sample_rate,wave_audio = wav.read(filename)

In [ ]:
wave_sample_rate

In [ ]:
wave_audio

In [ ]:
data

In [ ]:
import pandas as pd
metadata = pd.read_csv('../input/urbansound8k/UrbanSound8K.csv')

In [ ]:
metadata.head()

In [ ]:
filename = '../input/urbansound8k/fold5/100263-2-0-117.wav'
plt.figure(figsize=(14,5))
data,sample_rate=librosa.load(filename)
librosa.display.waveplot(data,sr=sample_rate)
ipd.Audio(filename)

In [ ]:
metadata['class'].value_counts()

# AUDIO CLASSIFICATION DATA PREPROCESSING

In [ ]:
import librosa
audio_file_path = '../input/urbansound8k/fold2/102858-5-0-1.wav'
librosa_audio_data,librosa_sample_rate = librosa.load(audio_file_path)

In [ ]:
print(librosa_audio_data)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,4))
plt.plot(librosa_audio_data)

In [ ]:
from scipy.io import wavfile as wav
wave_sample_rate,wave_audio = wav.read(audio_file_path)

In [ ]:
wave_audio

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(wave_audio)

MEL FREQUENCY CEPSTRAL COEFFICIENT

In [ ]:
mfccs = librosa.feature.mfcc(y=librosa_audio_data,sr=librosa_sample_rate,n_mfcc=40)

In [ ]:
mfccs

In [ ]:
import pandas as pd
import os
audio_dataset_path = '../input/urbansound8k/'
metadata = pd.read_csv('../input/urbansound8k/UrbanSound8K.csv')
metadata.head()

In [ ]:
def feature_extractor(file):
    audio,sample_rate = librosa.load(file_name,res_type='kaiser_fast')
    mfccs_feature = librosa.feature.mfcc(y=audio,sr=sample_rate,n_mfcc=40)
    mfccs_scaled_feature = np.mean(mfccs_feature.T,axis=0)
    return mfccs_scaled_feature

In [ ]:
import numpy as np
from tqdm import tqdm
extracted_features = []
for index_num,row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path),'fold'+str(row["fold"])+'/',str(row['slice_file_name']))
    final_class_labels = row['class']
    data= feature_extractor(file_name)
    extracted_features.append([data,final_class_labels])

In [ ]:
extracted_features_df = pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()

In [ ]:
x = np.array(extracted_features_df['feature'].tolist())
y = np.array(extracted_features_df['class'].tolist())

In [ ]:
y = np.array(pd.get_dummies(y))

In [ ]:
y

In [ ]:
y.shape

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=101)

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
x_train[5]

In [ ]:
y_train[5]

In [ ]:
extracted_features_df['class'][5]

# MODEL CREATION

In [ ]:
import tensorflow

In [ ]:
tensorflow.__version__

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from sklearn import metrics

In [ ]:
num_labels = y.shape[1]

In [ ]:
model = Sequential()
model.add(Dense(100,input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer = 'adam')

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime

num_epochs=100
num_batch_size=32

checkpointer = ModelCheckpoint(filepath='./audio_classification.hdf5',
                              verbose=1,save_best_only=True)
start = datetime.now()
model.fit(x_train,y_train,batch_size=num_batch_size,epochs=num_epochs,validation_data=(x_test,y_test),callbacks=[checkpointer])
duration = datetime.now() - start
print('Training time is ',duration)

In [ ]:
test_accuracy = model.evaluate(x_test,y_test)
print(test_accuracy[1])

In [ ]:
fname = '../input/urbansound8k/fold6/104327-2-0-26.wav'


In [ ]:
ipd.Audio(fname)

In [ ]:
prediction_feature = feature_extractor(fname)
prediction_feature = prediction_feature.reshape(1,-1)

In [ ]:
model.predict_classes(prediction_feature)

In [ ]:
metadata['class'].unique()

In [ ]:
def predict_sound_source(fname):
    prediction_feature = feature_extractor(fname)
    prediction_feature = prediction_feature.reshape(1,-1)
    a =  model.predict_classes(prediction_feature)
    if a==np.array(1):
        return 'Children playing'
    elif a==np.array(0):
        return 'Dog Barking'
    elif a== np.array(2):
        return 'Car Horn'
    elif a==np.array(3):
        return 'AC'
    elif a==np.array(4):
        return 'Street Music'
    elif a==np.array(5):
        return 'Gun Shot'
    elif a==np.array(6):
        return 'Siren'
    elif a==np.array(7):
        return 'ENgine Idling'
    elif a==np.array(8):
        return 'Jack Hammer'
    else:
        return 'drilling'
    

    
    

In [ ]:
ipd.Audio('../input/urbansound8k/fold2/102871-8-0-4.wav')



In [ ]:
predict_sound_source('../input/urbansound8k/fold2/102871-8-0-4.wav')

In [ ]:
ipd.Audio('../input/urbansound8k/fold6/104327-2-0-15.wav')